<a href="https://colab.research.google.com/github/chopstickexe/deep-learning-from-scratch-2/blob/master/ch07_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

$$
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
$$

## 数式の表記

変数: 小文字イタリック $x$

定数: 大文字イタリック $X$

ベクトル: 小文字ローマン体太字 $\vect{x}$

行列: 大文字ローマン体太字 $\mat{X}$

## 公式実装のclone

In [1]:
!git clone --depth=1 https://github.com/oreilly-japan/deep-learning-from-scratch-2.git
import sys 
sys.path.append('deep-learning-from-scratch-2')

Cloning into 'deep-learning-from-scratch-2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 73 (delta 13), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


# 7章 RNNによる文章生成

6章で実装した言語モデルが出力する確率分布に従って単語をサンプリングし，文章を生成する．

## 7.1 言語モデルを使った文章生成

In [2]:
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
  def generate(self, start_id, skip_ids=None, sample_size=100): 
    '''
    start_id: 開始単語のID
    skip_ids: <unk>など生成に使いたくない単語のリスト
    sample_size=文長
    '''
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size:
      x = np.array(x).reshape(1, 1)
      score = self.predict(x)  # Softmaxレイヤの直前の出力
      p = softmax(score.flatten())  # flatten(): ndarrayを一次元の配列化する

      sampled = np.random.choice(len(p), size=1, p=p)  # サンプリング．len(p)は生成されるサンプルの値域最大値，sizeは生成されるサンプル数，pはサンプルする際に使われる確率分布

      if (skip_ids is None) or (sampled not in skip_ids):
        x = sampled
        word_ids.append(int(x))

    return word_ids

In [5]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('deep-learning-from-scratch-2/ch06/Rnnlm.pkl')

# start文字とskip文字の設定
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 文章生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you were buying it and he is n't gorky.
 and some people addressed an advantages of the defense executive its think jobs in its purchases over because it 's returned to make it by presence or these companies said the express institute of strategy officials said.
 other person hit the previously filing with the segments individual cuts of powers processing.
 an even separate cowboys also said has sold next new suitor for office owner of columbia securities yesterday today.
 in the transportation pension fund a agency suffered only three fasb acquisitions.
 mr. jones is closely
